In [1]:
import sys
import qiskit
import qiskit_aer
import qiskit_ibm_runtime
print("Python version:", sys.version)
print("Qiskit version:", qiskit.__version__)
print("Qiskit-Aer version:", qiskit_aer.__version__)
print("Qiskit-IBM-Runtime version:", qiskit_ibm_runtime.__version__)
print("Kernel:", sys.executable)

Python version: 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
Qiskit version: 1.3.2
Qiskit-Aer version: 0.16.0
Qiskit-IBM-Runtime version: 0.34.0
Kernel: C:\code\envs\qiskit_env\Scripts\python.exe


In [4]:
#!/usr/bin/env python3
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# Settings for CHSH experiment (angles in radians for measurements)
angles = [
    (0, np.pi / 8),         # 0° and 22.5°
    (0, 3 * np.pi / 8),     # 0° and 67.5°
    (np.pi / 4, np.pi / 8), # 45° and 22.5°
    (np.pi / 4, 3 * np.pi / 8)  # 45° and 67.5°
]


# If using hardware, specify the backend name (optional)
backend_name = 'ibm_brisbane'

def run_on_simulator(shots=128):
    # Step 1: Set up the simulator
    simulator = AerSimulator()
    print("Running on simulator:", simulator.name)

    # Step 2: Create and run circuits
    expectation_values = []
    for theta1, theta2 in angles:
        # Create a new circuit for each angle pair
        qc = QuantumCircuit(2)  # 2 qubits
        qc.h(0)                 # Hadamard gate on qubit 0 (superposition)
        qc.cx(0, 1)             # CNOT gate to entangle qubits 0 and 1
        qc.ry(-2 * theta1, 0)   # Rotate qubit 0’s measurement basis
        qc.ry(-2 * theta2, 1)   # Rotate qubit 1’s measurement basis
        qc.measure_all()        # Measure both qubits

        # Run the circuit
        job = simulator.run(qc, shots=shots)
        result = job.result()
        counts = result.get_counts()  # Get measurement outcomes (e.g., '00', '11')

        # Step 3: Calculate expectation value from counts
        total_shots = sum(counts.values())
        expectation = 0
        for outcome, count in counts.items():
            # Parity: +1 for '00' or '11', -1 for '01' or '10'
            parity = (-1) ** (int(outcome[0]) + int(outcome[1]))
            expectation += parity * count
        expectation = expectation / total_shots
        expectation_values.append(expectation)

    # Step 4: Compute and print CHSH value
    S = expectation_values[0] - expectation_values[1] + expectation_values[2] + expectation_values[3]
    print(f"CHSH S-value: {S:.4f}")

def run_on_hardware(shots=128):
    # Step 1: Connect to IBM Quantum
    try:
        service = QiskitRuntimeService(instance='ibm-q/open/main')
        # print([backend.name for backend in service.backends()])  # See which backends are available 
        backend = service.backend(backend_name)
    except Exception as e:
        print("Error connecting to IBM Quantum:", e)
        exit()

    print("Running on hardware:", backend.name)

    # Step 2: Create circuits
    circuits = []
    for theta1, theta2 in angles:
        qc = QuantumCircuit(2)  # 2 qubits
        qc.h(0)                 # Hadamard gate on qubit 0
        qc.cx(0, 1)             # CNOT gate to entangle
        qc.ry(-2 * theta1, 0)   # Rotate qubit 0
        qc.ry(-2 * theta2, 1)   # Rotate qubit 1
        qc.measure_all()        # Measure both qubits
        circuits.append(qc)

    # Step 3: Transpile circuits to match hardware gates
    transpiled_circuits = transpile(circuits, backend=backend, optimization_level=1)

    # Step 4: Set up the Sampler for hardware
    sampler = Sampler(mode=backend)
    sampler.options.default_shots = shots

    # Step 5: Run all circuits at once
    job = sampler.run(transpiled_circuits)
    print("Job ID:", job.job_id())
    result = job.result()

    # Step 6: Calculate expectation values from results
    expectation_values = []
    for i in range(len(angles)):
        # Get counts from the result (Sampler returns a list of PubResults)
        data = result[i].data
        counts = data.meas.get_counts()  # 'meas' is the classical register name from measure_all()

        # Calculate expectation value
        total_shots = sum(counts.values())
        expectation = 0
        for outcome, count in counts.items():
            # Parity: +1 for '00' or '11', -1 for '01' or '10'
            parity = (-1) ** (int(outcome[0]) + int(outcome[1]))
            expectation += parity * count
        expectation = expectation / total_shots
        expectation_values.append(expectation)

    # Step 7: Compute and print CHSH value
    S = expectation_values[0] - expectation_values[1] + expectation_values[2] + expectation_values[3]
    print(f"CHSH S-value: {S:.4f}")

if __name__ == "__main__":
    # Change this to False to run on hardware
    use_simulator = False
    
    shots = 128

    if use_simulator:
        run_on_simulator(shots=shots)
    else:
        run_on_hardware(shots=shots)
    print(np.sqrt(2)*2)

Running on hardware: ibm_brisbane
Job ID: d0tspx34mb60008xey5g
CHSH S-value: 2.6406
2.8284271247461903
